## IMPORTATION DES BIBLIOTHEQUES ET PARAMETRES GLOBAUX

In [ ]:
import os 
import pandas as pd 
import numpy as np
import re
import matplotlib.pyplot as plt
import geopandas as gpd
import importlib
import time
from datetime import datetime
from matplotlib.colors import LinearSegmentedColormap

%matplotlib qt
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']
plt.rcParams.update({'font.size': 20})

pd.set_option('display.max_rows', 100)

# Obtenir la date du jour dans le format jjmmaaa
date_du_jour = datetime.now().strftime("%d%m%y")

# Afficher les milliers avec un espace
pd.options.display.float_format = '{: .2f}'.format

path = 'C:/Users/anormand/Documents/Projet Python/Biodiv/Data'  # Remplace par le chemin vers ton fichier

# Créer le chemin dynamique
save_path = f'C:/Users/anormand/Documents/Projet Python/Biodiv/Resultats/results_{date_du_jour}'

# Vérifiez si le répertoire n'existe pas déjà
if not os.path.exists(save_path):
    # Si le répertoire n'existe pas, créez-le
    os.makedirs(save_path)
    print("Répertoire créé avec succès à", save_path)
else:        print("Le répertoire existe déjà à", save_path)


In [25]:
### Importation des fonctions définies dans les fichiers biodiv ###
import affichage_carte_biodiv
importlib.reload(affichage_carte_biodiv)
from affichage_carte_biodiv import (
    configurer_carte,
    ajouter_couche_SIG,
    ajouter_couche_continue,
    ajouter_couche_discrete,
    ajouter_couche_statut,
    ajouter_couche_point,
    afficher_carte_defaut
)
import normalisation_biodiv
importlib.reload(normalisation_biodiv)
from normalisation_biodiv import (
    normaliser_par_maille,
    normaliser_par_maille_et_clade,
    normaliser_par_espece,
    normaliser_par_clade,
    normaliser_unique,
    normaliser_log,
    normaliser_par_periode
)
import exploration_biodiv
importlib.reload(exploration_biodiv)
from exploration_biodiv import (
    filtrer_top,
    afficher_top_especes,
    chercher_espece,
    explorer_clade,
    chercher_especes_protegees
)
import correlation_prediction
importlib.reload(correlation_prediction)
from correlation_prediction import (
    calculer_matrice_correlation,
    calculer_correlation_sujet,
    recalculer_nombreObs_par_correlation,
    calculer_prediction,
    calculer_seuil,
    recherche_espece_absente
)
import biodiversite_endemisme_biodiv
importlib.reload(biodiversite_endemisme_biodiv)
from biodiversite_endemisme_biodiv import (
    calculer_shannon,
    calculer_simpson,
    calculer_WE,
    calculer_indices
)
import clustering_geo_biodiv
importlib.reload(clustering_geo_biodiv)
from clustering_geo_biodiv import (
    analyser_composantes_principales,
    former_cluster_biogeo,
    determiner_k,
    etudier_composition_cluster
)
import clustering_espece_biodiv
importlib.reload(clustering_espece_biodiv)
from clustering_espece_biodiv import (
    generer_dendogram,
    former_cluster_espece,
    chercher_numcluster_espece,
    lister_especes_dans_cluster,
    grouper_dans_maille,
    etude_cluster_local,
    chercher_especes_pas_presentes,
    chercher_zone_espece_pas_presente
)
import fonctions_annexes_biodiv
importlib.reload(fonctions_annexes_biodiv)
from fonctions_annexes_biodiv import (
    round_to_sig,
    generer_dictionnaire_taxonomie,
    afficher_dataframe,
    completer_df,
    lister_mailles_dans_site,
    ajouter_nom_site_df
)

import evolution_temporelle
importlib.reload(evolution_temporelle)
from evolution_temporelle import (
    suivre_disparition_geo,
    determiner_statut
)

## IMPORTATION DES DONNEES

In [ ]:
### Importer les données par groupe ###
path='C:/Users/anormand/Documents/Projet Python/Biodiv/Data/'
cle_ID='cdRef'
cle_geo='codeMaille10Km'

# Chargement des donnees géographiques
# Charger les données géographiques des mailles 10 km
mailles_fp = path+r'SIG\L93_10X10.shp'  # Remplace par le chemin vers ton fichier
carte_maille = gpd.read_file(mailles_fp)
nouveaux_noms_columns = {'CD_SIG': 'codeMaille10Km'}
carte_maille = carte_maille.rename(columns=nouveaux_noms_columns)

bioregion_fichier = r'C:\Users\anormand\Documents\Projet Python\Biodiv\Data\SIG\region_biogeographique.shp'  # Remplace par le chemin vers ton fichier
bioregion_gpd = gpd.read_file(bioregion_fichier)

departement_fichier = r'C:\Users\anormand\Documents\Projet Python\Biodiv\Data\SIG\carte_departements.geojson'  # Remplace par le chemin vers ton fichier
departement_gpd = gpd.read_file(departement_fichier)

PNR_fichier = r'C:\Users\anormand\Documents\Projet Python\Biodiv\Data\SIG\N_ENP_PNR_S_000.shx'  # Remplace par le chemin vers ton fichier
PNR_gpd = gpd.read_file(PNR_fichier)

PN_fichier = r'C:\Users\anormand\Documents\Projet Python\Biodiv\Data\SIG\N_ENP_PN_S_000.shx'  # Remplace par le chemin vers ton fichier
PN_gpd = gpd.read_file(PN_fichier)

PN_et_PNR_gpd = pd.concat([PNR_gpd, PN_gpd], axis=1)

# Chargement des donnees d'observation
dossier='format_maille_espece_periode_1800_1990_2010_2024/'
# Création du DataFrame
df_inpn_periode = pd.DataFrame()
# Nom des groupes
groupes= ['mammiferes', 'crustaces', 'escargots', 'fonges', 'reptiles', 'insectes','poissons','oiseaux','plantes','autres']

for groupe in groupes:
    # Charger les données de la classe
    df_temp = pd.read_csv(path+dossier+'data_INPN_'+cle_geo+'_periodes_'+groupe+'.csv',dtype=str)
    df_temp['nombreObs']=df_temp['nombreObs'].astype(float)
    # Ajouter le DataFrame à la liste des DataFrames
    df_inpn_periode = pd.concat([df_inpn_periode, df_temp], ignore_index=True)

# garder seulement les données correspodnant à la france metropolitaine
if cle_geo=='codeMaille10Km':
    df_inpn_france=df_inpn_periode[df_inpn_periode['codeMaille10Km'].isin(carte_maille['codeMaille10Km'])]

if cle_geo=='codeDepartement':
    df_inpn_periode.rename(columns={'codeInseeDepartement': 'codeDepartement'}, inplace=True)
    departement_gpd.rename(columns={'code': 'codeDepartement'}, inplace=True)
    df_inpn_france=df_inpn_periode[df_inpn_periode['codeDepartement'].isin(departement_gpd['codeDepartement'])]

# Garder seulement le premier nom Vernaculaire
df_inpn_france.loc[:, 'nomVernaculaire'] = df_inpn_france['nomVernaculaire'].str.split(',').str[0]

#generer le dicotionnaire d'espece
dico_taxo=generer_dictionnaire_taxonomie(df_inpn_france,cle_ID)

# permet de garder seulement les variables que l'on souhaite
df_inpn_periode = df_inpn_france.groupby([cle_geo,cle_ID,'periode'],as_index=False).agg({'nombreObs': 'sum'})
df_inpn_periode=pd.merge(df_inpn_periode,dico_taxo,on=cle_ID)

df_inpn_sansperiode = df_inpn_france.groupby([cle_geo,cle_ID],as_index=False).agg({'nombreObs': 'sum'})
df_inpn_sansperiode=pd.merge(df_inpn_sansperiode,dico_taxo,on=cle_ID)

NORMALISATION DES DONNEES

In [48]:
### Filtrage et normalisation des données ###

#Choix du dataframe à utiliser
df_biodiv=df_biodiv_sansperiode.copy()

#taux_min=0.5
#df_inpn=ajouter_nom_site_df(df_biodiv,carte_maille,PN_et_PNR_gpd,taux_min,cle_geo)

# filtrer les règnes
df_biodiv=df_biodiv[(df_biodiv['kingdom']=='Animalia')|(df_biodiv['kingdom']=='Plantae')|(df_biodiv['kingdom']=='Fungi')]
df_biodiv=df_biodiv[(df_biodiv['kingdom']=='Animalia')|(df_biodiv['kingdom']=='Plantae')]

# Normalisations des données
df_biodiv=normaliser_unique(df_biodiv)
df_biodiv=normaliser_par_espece(df_biodiv,cle_ID)
df_biodiv=normaliser_par_maille_et_clade(df_biodiv, cle_geo, clade_col='kingdom', observation_col='nombreObs')
#df_biodiv=normaliser_par_maille(df_biodiv, cle_geo='NOM_SITE', observation_col='nombreObs')
df_biodiv=normaliser_log(df_biodiv,'nombreObs_norm_par_maille_et_kingdom')
"""
df_inpn=normaliser_par_periode(df_inpn)
df_inpn=normaliser_par_periode(df_inpn,'nombreObs_norm_par_espece')
df_inpn=normaliser_par_periode(df_inpn,'nombreObs_norm_par_maille_et_regne')
"""
liste_nombres=[col for col in df_biodiv.columns if 'nombreObs' in col]

Le nombre d observations total par espèce est fixé à 10 000
Il y a en moyenne 111732.1 observations par kingdom par maille


## EXPLORATION DES DONNEES  

In [ ]:
# Créer des filtres prédéfinis d'espèces
print('arbres')
df_arbres=df_inpn[(df_inpn['classe']=='Pinopsida')|(df_inpn['ordre']=='Fagales')|(df_inpn['genre']=='Crataegus')|(df_inpn['genre']=='Prunus')|(df_inpn['genre']=='Malus')|(df_inpn['genre']=='Sorbus')|(df_inpn['genre']=='Pyrus')]

print('plantes')
df_plantes=df_inpn[(df_inpn['regne']=='Plantae')]

print('mammifères')
df_mammifères=df_inpn[(df_inpn['classe']=='Mammalia')]

print('poissons')
df_poissons=df_inpn[(df_inpn['classe']=='Actinopterygii')|(df_inpn['classe']=='Myxini')|(df_inpn['classe']=='Leptocardii')|
                    (df_inpn['classe']=='Holocephali')|(df_inpn['classe']=='Dipneusti')|(df_inpn['classe']=='Petromyzonti')|(df_inpn['classe']=='Elasmobranchii')]

print('oiseaux')
df_oiseaux=df_inpn[(df_inpn['classe']=='Aves')]

print('reptiles')
df_reptiles=df_inpn[(df_inpn['ordre']=='Chelonii')|(df_inpn['ordre']=='Squamata')|(df_inpn['ordre']=='Crocodylia')]

print('arthropodes')
df_arthropodes=df_inpn[(df_inpn['classe']=='Insecta')|(df_inpn['classe']=='Arachnida')|
                        (df_inpn['classe']=='Malacostraca')|(df_inpn['classe']=='Branchiopoda')|
                        (df_inpn['classe']=='Chilopoda')|(df_inpn['classe']=='Diplopoda')]

print('graminees')
df_graminees=df_inpn[(df_inpn['famille']=='Poaceae')]

print('mousses')
df_mousses=df_inpn[(df_inpn['ordre']=='Sphagnales')|(df_inpn['ordre']=='Hypnales')|
                    (df_inpn['ordre']=='Dicranales')|(df_inpn['ordre']=='Bryales')|
                    (df_inpn['ordre']=='Buxbaumiales')|(df_inpn['ordre']=='Diphysciales')|
                    (df_inpn['ordre']=='Grimmiales')|(df_inpn['ordre']=='Andreaeales')|
                    (df_inpn['ordre']=='Polytrichales')|(df_inpn['ordre']=='Hookeriales')]

print('amphibiens')
df_amphibiens=df_inpn[(df_inpn['classe']=='Amphibia')]

print('mollusques')
df_mollusques=df_inpn[(df_inpn['classe']=='Gastropoda')|(df_inpn['classe']=='Bivalvia')|
                    (df_inpn['classe']=='Cephalopoda')|(df_inpn['classe']=='Monoplacophora')|
                    (df_inpn['classe']=='Scaphopoda')]


print('Araignées')
df_araignées=df_inpn[(df_inpn['ordre']=='Araneae')]

print('Champignons')
df_champignons=df_inpn[(df_inpn['regne']=='Fungi')]

print('Papillons')
df_papillons=df_inpn[(df_inpn['ordre']=='Lepidoptera')]


In [62]:
# Afficher le top des espèces les plus observées
print('nombre d"espèces observées :',len(df_biodiv[cle_ID].unique()))
col_valeur='nombreObs'
top_espece=afficher_top_especes(df_biodiv,dico_taxo,col_valeur,cle_ID)
afficher_dataframe(top_espece,[col_valeur,cle_ID,'species','family','order','class','kingdom'],col_valeur).head(50)

nombre d"espèces observées : 30390


,nombreObs,speciesKey,species,family,order,class,kingdom
0,140728642,2674642,Holomitrium williamsii,Dicranaceae,Dicranales,Bryopsida,Plantae
1,105546475,2674688,Holomitrium arboreum,Dicranaceae,Dicranales,Bryopsida,Plantae
2,35141167,2669226,Sphagnum sancto-josephense,Sphagnaceae,Sphagnales,Sphagnopsida,Plantae
3,35141085,2669884,Rhynchostegiopsis carolae,Leucomiaceae,Hookeriales,Bryopsida,Plantae
4,35140507,5279882,Daltonia ovalis,Daltoniaceae,Hookeriales,Bryopsida,Plantae
5,7359452,2481930,Cathartes aura,Cathartidae,Accipitriformes,Aves,Animalia
6,4055800,2480538,Buteo platypterus,Accipitridae,Accipitriformes,Aves,Animalia
7,3147847,9422220,Spatula discors,Anatidae,Anseriformes,Aves,Animalia
8,2871174,2480562,Buteo swainsoni,Accipitridae,Accipitriformes,Aves,Animalia
9,2181643,2498393,Dendrocygna autumnalis,Anatidae,Anseriformes,Aves,Animalia


In [ ]:
# Chercher les espèces à partir d'un mot 
col_valeur='nombreObs'
resultat_recherche=chercher_espece(df_inpn,dico_taxo,'Isopoda',col_valeur,cle_ID)
afficher_dataframe(resultat_recherche,[col_valeur,cle_ID,'nomVernaculaire','nomScientifique','famille','ordre','classe','regne','especeProtegee'],col_valeur).head(50)

In [ ]:
# Afficher les sous clades du clade choisi, le nombre d'espèces et le nombre d'observations
# Liste des clades = ['all','regne', 'classe', 'ordre', 'famille', 'genre','nomScientifique','nomVernaculaire']
clade = 'regne'
taxon='Fungi'

explorer_clade(df_inpn,clade,taxon)

In [ ]:
# Afficher la carte avec les mailles
afficher_carte_maille(carte_maille)

In [ ]:
# Listes prédéfinies de codes
liste_geo_PN_Calanques=['10kmL93E088N624','10kmL93E090N624',
             '10kmL93E087N623','10kmL93E088N623','10kmL93E089N623','10kmL93E090N623','10kmL93E091N623',
             '10kmL93E086N622','10kmL93E087N622','10kmL93E088N622','10kmL93E089N622','10kmL93E090N622','10kmL93E091N622',
             '10kmL93E091N620','10kmL93E087N622','10kmL93E088N621','10kmL93E089N621','10kmL93E090N621','10kmL93E091N621',
            ]
liste_geo_PNR_Vosges=['10kmL93E100N680','10kmL93E101N680',
             '10kmL93E100N679','10kmL93E101N679','10kmL93E098N678','10kmL93E099N678','10kmL93E100N678','10kmL93E101N678',
             '10kmL93E098N677','10kmL93E099N677','10kmL93E100N677','10kmL93E101N677',
             '10kmL93E098N676','10kmL93E099N676','10kmL93E100N676','10kmL93E101N676',
             '10kmL93E098N675','10kmL93E099N675','10kmL93E100N675','10kmL93E097N675',
            ]
liste_geo_PN_Cevennes=['10kmL93E073N632', '10kmL93E073N633', '10kmL93E073N634', '10kmL93E073N635', '10kmL93E073N636', '10kmL93E073N637',
            '10kmL93E074N632', '10kmL93E074N633', '10kmL93E074N634', '10kmL93E074N635', '10kmL93E074N636', '10kmL93E074N637',
            '10kmL93E075N632', '10kmL93E075N633', '10kmL93E075N634', '10kmL93E075N635', '10kmL93E075N636', '10kmL93E075N637',
            '10kmL93E076N632', '10kmL93E076N633', '10kmL93E076N634', '10kmL93E076N635', '10kmL93E076N636', '10kmL93E076N637',
            '10kmL93E077N632', '10kmL93E077N633', '10kmL93E077N634', '10kmL93E077N635', '10kmL93E077N636', '10kmL93E077N637',
            '10kmL93E078N635','10kmL93E079N635','10kmL93E079N636','10kmL93E078N636'
            ]

In [ ]:
# Filtrer une liste de mailles à étudier
liste_codes=['10kmL93E089N623']
col_valeur='nombreObs_norm_par_maille_et_regne'

df_filt=df_inpn[df_inpn[cle_geo].isin(liste_codes)]
df_maille=df_filt.groupby(cle_ID)[col_valeur].sum()

df_dico=generer_dictionnaire_taxonomie(df_filt,cle_ID)
df_maille=pd.merge(df_maille,df_dico,on=cle_ID)

afficher_dataframe(df_maille,[col_valeur,cle_ID,'nomVernaculaire','nomScientifique','famille','ordre','classe','regne','especeProtegee'],col_sort=col_valeur).head(10)

In [ ]:
# Rechercher les espèces protégées présentes dans une zone
col_valeur='nombreObs_norm_par_espece'
liste_codes=liste_geo_PNR_Vosges

chercher_especes_protegees(df_inpn,liste_codes,cle_geo='codeMaille10Km',cle_ID='cdRef',col_valeur=col_valeur)

afficher_dataframe(grouped_local_especes_protegee,[col_valeur,cle_ID,'nomVernaculaire','nomScientifique','famille','ordre','classe','regne','especeProtegee']).head(10)

## AFFICHAGE DES DONNEES SOUS FORME DE CARTE 

In [ ]:
import affichage_carte_biodiv
importlib.reload(affichage_carte_biodiv)
from affichage_carte_biodiv import (
    configurer_carte,
    ajouter_couche_SIG,
    ajouter_couche_continue,
    ajouter_couche_discrete,
    ajouter_couche_statut,
    ajouter_couche_point,
    afficher_carte_defaut
)
# Afficher les données sous forme de carte
col_valeur='nombreObs_norm_par_maille_et_kingdom'
colormap='viridis'

clade = 'species' #clade can be :  nomScientifique 	nomVernaculaire  regne 	classe 	ordre 	famille  genre 
taxon='Sturnus vulgaris'
titre=clade+'_'+taxon+' - var '+col_valeur
df_filt=df_biodiv[(df_biodiv[clade]==taxon)]

""" basemap_type :
'OpenStreetMap': ctx.providers.OpenStreetMap.France,
'GeoportailSatellite': ctx.providers.GeoportailFrance.orthos,
"""

fig, ax=configurer_carte('GeoportailSatellite',13,9,zoom=6,xlim_min=xlim_min,xlim_max=xlim_max,ylim_min=ylim_min,ylim_max=ylim_max)

#fig, ax=ajouter_couche_SIG(fig, ax,bioregion_gpd)
fig, ax=ajouter_couche_SIG(fig, ax, departement_gpd,edgecolor="white",facecolor='none',linewidth=0.5,linestyle='--',alpha=1,with_label=True,col_label='code',label_color='white',fontsize=8)
fig, ax=ajouter_couche_continue(fig, ax, df_filt,carte_maille,col_valeur,'codeMaille10Km',quantile_inf=0.01,quantile_sup=0.95,cmap_choice='viridis',val_alpha=0.8)
#fig, ax=ajouter_couche_SIG(fig, ax, PNR_gpd,edgecolor="white",facecolor='yellow',linewidth=0.5,linestyle='--',alpha=0.3)
#fig, ax=ajouter_couche_SIG(fig, ax, PN_gpd,edgecolor="white",facecolor='lightgreen',linewidth=0.5,linestyle='--',alpha=0.5)
ax.set_title(titre, fontsize=16)  # Taille de la police définie à 16
fig.savefig(save_path+'/'+titre+'.png', dpi=300, bbox_inches='tight')  # Enregistre au format PNG avec une résolution de 300 DPI

plt.show()

## INDICE DE BIODIVERSITE et ENDEMISME  

In [ ]:
# Calculer les indices de biodiversité et d'endémisme
col_valeur='nombreObs_norm_par_maille_et_kingdom'
colormap='plasma'
cle_geo_etude='codeMaille5Km'
df_etude=df_biodiv
df_etude=df_etude[df_etude[cle_geo_etude].notna()]
#df_etude=df_etude[df_etude['habitatEspeceTaxref'].str.contains("marin",na=False,case=False)]
df_etude=filtrer_top(df_etude,'nombreObs',100,cle_ID)

df_indice=calculer_indices(df_etude,col_valeur,cle_geo_etude,cle_ID)
fond_carte='GeoportailSatellite'


In [ ]:
nom='totale'
#Indices de biodiversité : shannon ou simpson ou nb espèces
indice_choisi="indice_d_endemisme"
titre='Biodiversité_'+nom+'_'+indice_choisi

fig, ax=configurer_carte('OpenStreetMap',13,9,xlim_min=xlim_min,xlim_max=xlim_max,ylim_min=ylim_min,ylim_max=ylim_max,zoom=8)
fig, ax=ajouter_couche_continue(fig, ax, df_indice,carte_maille,indice_choisi,cle_geo,
                                quantile_inf=0.02,quantile_sup=0.99, borne_min=None,borne_max=None,
                                cmap_choice='plasma',val_alpha=0.6,colorbar_choice=True)
fig, ax=ajouter_couche_SIG(fig, ax,world_geo,linewidth=1)
ax.set_title(titre, fontsize=16)  # Taille de la police définie à 16
fig.savefig(save_path+'/'+titre+'.png', dpi=300, bbox_inches='tight')  # Enregistre au format PNG avec une résolution de 300 DPI

plt.show()

In [ ]:
df_geo_affichage=carte_maille
#PN_et_PNR_gpd['NOM_SITE'] = PN_et_PNR_gpd['NOM_SITE'].str.replace(r"\s*\[aire d'adhésion\]", "", regex=True, flags=re.IGNORECASE)
df_indice_affichage=df_indice
cle_geo_etude='codeMaille10Km'
nom='totale'
#Indices de biodiversité : shannon ou simpson ou nb espèces
indice_choisi="nombre_especes"
titre='Biodiversité_'+nom+'_'+indice_choisi
afficher_carte_defaut(titre,df_indice_affichage,df_geo=df_geo_affichage,couche_sig=world_geo,col_valeur=indice_choisi,cle_geo=cle_geo_etude,quantile_inf=0,quantile_sup=1,
                           cmap_choice=colormap,basemap_type=fond_carte,save_path=None)

indice_choisi='indice_de_Shannon' # Mesure la diversité en prenant en compte la richesse spécifique et l'abondance relative des espèces.
titre='Biodiversité_'+nom+'_'+indice_choisi
afficher_carte_defaut(titre,df_indice_affichage,df_geo=df_geo_affichage,couche_sig=world_geo,col_valeur=indice_choisi,cle_geo=cle_geo_etude,quantile_inf=0,quantile_sup=1,
                           cmap_choice=colormap,basemap_type=fond_carte,save_path=None)

indice_choisi='indice_de_Simpson' # Mesure la diversité en prenant en compte la richesse spécifique et l'abondance relative des espèces.
titre='Biodiversité_'+nom+'_'+indice_choisi
afficher_carte_defaut(titre,df_indice_affichage,df_geo=df_geo_affichage,couche_sig=world_geo,col_valeur=indice_choisi,cle_geo=cle_geo_etude,quantile_inf=0,quantile_sup=1,
                           cmap_choice=colormap,basemap_type=fond_carte,save_path=None)

df_indice_affichage.sort_values('indice_de_Simpson',ascending=False).reset_index(drop=True)


In [ ]:
df_indice_affichage.sort_values('indice_d_endemisme',ascending=False).reset_index(drop=True).tail(5)

In [ ]:
# Indice d'endémisme : WE
df_indice_affichage=df_indice
cle_geo_etude='NOM_SITE'
indice_choisi='indice_d_endemisme'
titre='Endémisme_'+nom+'_'+indice_choisi
afficher_carte_defaut(titre,df_indice_affichage,df_geo=df_geo_affichage,couche_sig=PN_et_PNR_gpd,col_valeur=indice_choisi,cle_geo=cle_geo_etude,quantile_inf=0,quantile_sup=0.96,
                           cmap_choice=colormap,basemap_type=fond_carte,save_path=None)

df_indice_affichage.sort_values('indice_d_endemisme',ascending=False).reset_index(drop=True)

In [ ]:
# Si on a fait des mailles pour étudier les parcs, attention plutot pour endemisme, pour le reste plutot passer directement par cle_geo ='NOM_SITE'
df_indice_parc=df_indice.reset_index()
df_indice_parc=ajouter_nom_site_df(df_indice_parc,carte_maille,PN_et_PNR_gpd,taux_min=0.5,cle_geo='codeMaille10Km',cle_nom_site='NOM_SITE',display=True,methode='exact')
df_indice_parc=df_indice_parc.groupby(['NOM_SITE'])[['nombre_especes','indice_de_Shannon','indice_de_Simpson','indice_d_endemisme']].mean().reset_index()

## ANALYSE PAR CLUSTERS BIOGEOGRAPHIQUES  

In [49]:
# Regrouper les données en clusters similaires et afficher la carte
#clade can be :  nomScientifique 	nomVernaculaire 	regne 	classe 	ordre 	famille 	genre 	
df_filt= filtrer_top(df_biodiv,'nombreObs',100,cle_ID)
col_valeur='nombreObs_norm_par_maille_et_kingdom_log'
choix_methode='ward' #method : kmeans ward

n_pca=10
k=8
df_pca=analyser_composantes_principales(df_filt,cle_geo,col_valeur,cle_ID,n_pca=n_pca)

#determinerk(df_pca)

df_cluster=former_cluster_biogeo(df_pca,method=choix_methode,k_cluster=k)

nombre d'espèces retenues dans le df :16199 (53%)
Variance expliquée par chaque composante: [0.12020329 0.05299062 0.03904496 0.02918799 0.02435018 0.0143472
 0.01118282 0.01048723 0.00955058 0.00871702]
Cluster
1    611
2    372
3    341
4    237
5    217
6    208
7     61
Name: count, dtype: int64


In [56]:
titre='Carte des biorégions du '+country.replace('_', '-')

"""
    'OpenStreetMap'
    'GeoportailSatellite'
"""
colormap='Accent_r' #Spectral_r plasma Paired (12) Set1_r (9) Set2_r (8) tab10 (10) Accent_r (8)
fig, ax=configurer_carte('OpenStreetMap',13,9,xlim_min=xlim_min,xlim_max=xlim_max,ylim_min=ylim_min,ylim_max=ylim_max,zoom=8)
#fig, ax=ajouter_couche_SIG(fig, ax,bioregion_gpd)

fig, ax=ajouter_couche_discrete(fig, ax,df_cluster,carte_maille,col_valeur='Cluster',cle_geo=cle_geo,cmap_choice=colormap,val_alpha=0.75,legend_choice=True)
#fig, ax=ajouter_couche_SIG(fig, ax,bioregion_gpd,linewidth=1)
#fig, ax=ajouter_couche_SIG(fig, ax,departement_gpd)
#fig, ax=ajouter_couche_SIG(fig, ax,eez_geo,linewidth=1,edgecolor='blue',linestyle='--')
fig, ax=ajouter_couche_SIG(fig, ax,world_geo,linewidth=1,edgecolor='black',linestyle='--')

ax.set_title(titre, fontsize=18)  # Taille de la police définie à 16

# Ajoute une ligne de texte en dessous de la figure
fig.text(0.5, 0.09, f'var : {col_valeur}, méthode k-means {choix_methode}', ha='center', va='center', fontsize=10)

fig.savefig(save_path+'/'+titre+'.png', dpi=300, bbox_inches='tight')  # Enregistre au format PNG avec une résolution de 300 DPI

plt.show()

In [27]:
# Afficher les espèces surreprésentés dans chaque cluster
#df_filt=creation_filtre_top(df_inpn,'nombreObs',2000)

cluster_composition=afficher_composition_cluster(df_filt,df_cluster,dico_taxo,'nombreObs_norm_par_espece',5,cle_ID,cle_geo)

"""    # Afficher les 5 espèces les plus présentes pour chaque cluster
    for cluster in clusters:
        cluster_df = sorted_df[sorted_df['Cluster'] == cluster]
        top_5_df = cluster_df.head(n)
        
        print(f"Cluster {cluster}:")
        for _, row in top_5_df[['nomVernaculaire', 'nomScientifique', '% composition']].iterrows():
            print(f"{row['nomVernaculaire']:<20} {row['nomScientifique']:<20} {row['% composition']:>10}")
        print("\n")"""

cluster_composition

,Cluster,speciesKey,nombreObs_norm_par_espece,kingdom,phylum,class,order,family,genus,species,totalObs,nombreObs_unique,% composition
5991,1,4284349,10000.01,Animalia,Chordata,NaN,Stephanoberyciformes,Melamphaidae,Scopelogadus,Scopelogadus bispinosus,18679714.78,1,0.10
8124,1,6521504,10000.01,Animalia,Mollusca,Cephalopoda,Oegopsida,Pyroteuthidae,Pterygioteuthis,Pterygioteuthis hoylei,18679714.78,1,0.10
1182,1,2290221,10000.00,Animalia,Mollusca,Cephalopoda,Oegopsida,Enoploteuthidae,Abraliopsis,Abraliopsis affinis,18679714.78,1,0.10
1577,1,2406027,10000.00,Animalia,Chordata,NaN,Myctophiformes,Myctophidae,Diogenichthys,Diogenichthys laternatus,18679714.78,1,0.10
6474,1,5213057,10000.00,Animalia,Chordata,NaN,Anguilliformes,Nemichthyidae,Avocettina,Avocettina bowersii,18679714.78,1,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71414,8,2480327,0.09,Animalia,Chordata,Aves,Charadriiformes,Charadriidae,Pluvialis,Pluvialis squatarola,38321774.12,1,0.00
77665,8,6065844,0.08,Animalia,Chordata,Aves,Charadriiformes,Laridae,Leucophaeus,Leucophaeus atricilla,38321774.12,1,0.00
71472,8,2481672,0.06,Animalia,Chordata,Aves,Charadriiformes,Scolopacidae,Limnodromus,Limnodromus griseus,38321774.12,1,0.00
71944,8,2669884,0.00,Plantae,Bryophyta,Bryopsida,Hookeriales,Leucomiaceae,Rhynchostegiopsis,Rhynchostegiopsis carolae,38321774.12,1,0.00


In [29]:
# Afficher les taxons les plus présents de chaque cluster
cluster_composition=afficher_composition_cluster(df_filt,df_cluster,dico_taxo,'nombreObs_norm_par_maille_et_kingdom',5,cle_ID,cle_geo)

cluster_composition

,Cluster,speciesKey,nombreObs_norm_par_maille_et_kingdom,kingdom,phylum,class,order,family,genus,species,totalObs,nombreObs_unique,% composition
1978,1,2480975,13087936.50,Animalia,Chordata,Aves,Suliformes,Sulidae,Sula,Sula leucogaster,151739658.53,1,8.60
1987,1,2481119,5380280.90,Animalia,Chordata,Aves,Charadriiformes,Laridae,Chlidonias,Chlidonias niger,151739658.53,1,3.50
1977,1,2480972,4958275.06,Animalia,Chordata,Aves,Suliformes,Sulidae,Sula,Sula sula,151739658.53,1,3.30
6547,1,5229344,4383011.35,Animalia,Chordata,Aves,Procellariiformes,Procellariidae,Puffinus,Puffinus pacificus,151739658.53,1,2.90
9304,1,8324617,4053953.62,Animalia,Chordata,Mammalia,Cetacea,Delphinidae,Delphinus,Delphinus delphis,151739658.53,1,2.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78489,8,7759294,0.04,Animalia,Chordata,Squamata,NaN,Colubridae,Drymarchon,Drymarchon corais,5845035.03,1,0.00
78823,8,8169649,0.04,Animalia,Chordata,Aves,Passeriformes,Parulidae,Leiothlypis,Leiothlypis celata,5845035.03,1,0.00
79367,8,9208233,0.04,Animalia,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Hamadryas,Hamadryas fornax,5845035.03,1,0.00
79947,8,10558140,0.04,Animalia,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Lycorea,Lycorea ilione,5845035.03,1,0.00


## ANALYSE PAR CLUSTERS D'ESPECES

In [58]:
# Filtrer les espèces les plus présentes pour réduire le temps de calcul
df_input=filtrer_top(df_biodiv,'nombreObs',100,cle_ID)

# info temps : seuil=10 000 - n=1680 => t=60s
# pour toutes les espèces (16 792) => t=6000s 
# Cas général : t = n^2 / 47040 ou n=racine(t*47040)

nombre d'espèces retenues dans le df :4837 (16%)


In [61]:
# Calculer la matrice de corrélation et affichage du dendogram
col_values_mr='nombreObs_norm_par_maille_et_kingdom'
mat_corr_mr=calculer_matrice_correlation(df_input,col_values_mr,'pearson',cle_geo,cle_ID)
mat_corr_mr= mat_corr_mr.astype(float)
Z_mr=generer_dendogram(mat_corr_mr,methode='ward',display=0) #plutot ward ou complete

"""
col_values_mrl='nombreObs_normalisé_mrl'
corr_mrl=correlation_matrix_dendogram(df_input,col_values_mrl)
Z_mrl=dendogram(corr_mrl,methode='ward',display=0) #plutot ward ou complete

col_values_obs='nombreObs'
corr_obs=correlation_matrix_dendogram(df_input,col_values_obs)
Z_obs=dendogram(corr_obs,methode='ward',display=0)

col_values_unique='ObsUnique'
corr_unique=correlation_matrix_dendogram(df_input,col_values_unique)
Z_unique=dendogram(corr_unique,methode='ward',display=0)

corr_moy=(corr_mr+corr_unique+corr_obs)/3
Z_moy=dendogram(corr_moy,methode='ward',display=0)
"""

"\ncol_values_mrl='nombreObs_normalisé_mrl'\ncorr_mrl=correlation_matrix_dendogram(df_input,col_values_mrl)\nZ_mrl=dendogram(corr_mrl,methode='ward',display=0) #plutot ward ou complete\n\ncol_values_obs='nombreObs'\ncorr_obs=correlation_matrix_dendogram(df_input,col_values_obs)\nZ_obs=dendogram(corr_obs,methode='ward',display=0)\n\ncol_values_unique='ObsUnique'\ncorr_unique=correlation_matrix_dendogram(df_input,col_values_unique)\nZ_unique=dendogram(corr_unique,methode='ward',display=0)\n\ncorr_moy=(corr_mr+corr_unique+corr_obs)/3\nZ_moy=dendogram(corr_moy,methode='ward',display=0)\n"

In [ ]:
# Sauvegarder les données
path='C:/Users/anormand/Documents/Projet Python/Biodiv/Data_intermed'

df_tosave = pd.DataFrame(mat_corr)
df_tosave.to_csv(path+'/corr_mr_ward_sansfungi.csv', index=False)


In [ ]:
# Importer les données
path='C:/Users/anormand/Documents/Projet Python/Biodiv/Data_intermed'

Z_obs = pd.read_csv(path+ '/Z_obs_sansfungi.csv',dtype=str) # ward

In [ ]:
# Former les clusters en fonction du niveau choisi
col_valeur='nombreObs_norm_par_maille_et_regne'
Z=Z_mr
lvl=8
criterion='distance'

df_corr_cluster=former_cluster_espece(df_input, Z, col_valeur=col_valeur, level=lvl, crit='distance')
df_input_clustered=pd.merge(df_input,df_corr_cluster[[cle_ID,'Cluster_corr']],on=cle_ID)

print('nombre de cluster formés :'+str(len(df_corr_cluster['Cluster_corr'].unique())))

In [ ]:
# Chercher le cluster contenant une espèce en particulier et afficher les espèces qu'il contient, ordonne selon la colonne col_values
espece='Sus scrofa'
col_valeur='nombreObs_norm_par_maille_et_regne'
df_corr_cluster=df_corr_cluster
#Recherche du cluster contenant l'espèce
num_cluster=chercher_numcluster_espece(df_corr_cluster,espece)

# Affichage du cluster ordonné par col_values
liste_especes_cluster_choisi=lister_especes_dans_cluster(df_input_clustered,num_cluster,col_valeur='nombreObs_norm_par_maille_et_regne')

print('Le cluster regroupe '+str(len(liste_especes_cluster_choisi))+' espèces')

afficher_dataframe(liste_especes_cluster_choisi,[cle_ID,'nomScientifique','nomVernaculaire','ordre','classe','regne',col_valeur],col_sort=col_valeur).head(100)

In [ ]:
# Afficher l'aire de répartition correspondant au cluster selectionné
colormap='viridis'
col_valeur='nombreObs_norm_par_maille_et_regne'
titre=f'{col_valeur} cluster numéro {num_cluster}'
df_filt=df_input_clustered[df_input_clustered['Cluster_corr']==num_cluster]

# On récupère la liste unique des code_maille

afficher_carte_defaut(titre,df_filt,df_geo=carte_maille,couche_sig=departement_gpd,col_valeur=col_valeur,cle_geo=cle_geo,quantile_inf=0,quantile_sup=0.95,
                           cmap_choice=colormap,basemap_type=fond_carte,save_path=None)

In [ ]:
df_input_clustered

## ANALYSE AVANCEE PAR CLUSTER D'ESPECE

In [ ]:
# Remplir le dataframe df_inpn_cluster_espece en faisant correspondre maille et cluster

col_valeur='nombreObs_norm_par_maille_et_regne'
df_grouper_par_maille=grouper_dans_maille(df_input,df_corr_cluster,col_valeur)


In [ ]:
# Afficher la carte avec les mailles
afficher_carte_maille(carte_maille)

In [ ]:
# Rechercher les clusters présents dans une maille ou une liste de mailles, les ordonner selon la colonne col_values
col_choice='nombreObs_normalisé_par_espece' # pour les espèces les plus caractéristiques
liste_codes=liste_geo_PNR_Vosges

df_cluster_maille=etude_mailles(df_inpn_cluster_espece,liste_codes,['nombreObs','nombreObs_normalisé_par_maille_regne','nombreObs_normalisé_par_espece'],col_groupe='Cluster_corr')
df_dico=df_inpn_cluster_espece[['Cluster_corr', 'nomScientifique', 'nomVernaculaire']].drop_duplicates(subset=['Cluster_corr'])
df_cluster_maille = pd.merge(df_cluster_maille,df_dico, on='Cluster_corr', how='left')
df_cluster_maille = df_cluster_maille.sort_values(by=col_choice,ascending=False)
df_cluster_maille=df_cluster_maille.reset_index()

print(str(len(df_cluster_maille))+' groupes sont présents dans la zone étudiée')
affichage_dataframe(df_cluster_maille,['Cluster_corr','nomScientifique','nomVernaculaire','nombreObs_normalisé_par_espece','nombreObs','nombreObs_normalisé_par_maille_regne'],col_sort=col_choice).head(20)

In [ ]:
# Chercher le rang du cluster choisi dans la zone selectionnée
num_cluster=290
rank_cluster = df_cluster_maille[df_cluster_maille['Cluster_corr'] == num_cluster].index[0]
print(('Cluster rang ')+str(rank_cluster)+' dans la zone '+str(liste_codes))


In [ ]:
# Afficher les espèces présentes dans le cluster le plus représenté localement, 0 pour les espèces du cluster pas présentes localement
col_choice='nombreObs_normalisé_par_espece'

rank_cluster=1 #correspond au numéro du cluster : 0 = le premier

num_cluster=int(df_cluster_maille['Cluster_corr'].iloc[rank_cluster])
print(('Cluster n°')+str(num_cluster)+' dans la zone '+str(liste_codes))
cluster_local=etude_cluster_local(df_input,liste_codes,df_corr_cluster,num_cluster,col_choice)
affichage_dataframe(cluster_local,['nomScientifique','nomVernaculaire','classe','regne','nombreObs','nombreObs_normalisé_par_maille_regne','nombreObs_normalisé_par_espece',col_values],col_sort=col_choice)

In [ ]:
# Chercher les espèces qui pourraient potentiellement être présente dans la zone étudiée
# la liste est ordonnée de manière décroissante (espèces les plus adaptées en premier)

df_espece_pas_presentes=chercher_especes_pas_presentes(df_input,df_cluster_maille,df_corr_cluster,liste_codes)
affichage_dataframe(df_espece_pas_presentes,['nomScientifique','nomVernaculaire','genre','famille','ordre','classe','regne','Cluster_corr','nombreObs']).head(20)

In [ ]:
# chercher les zones les plus adaptées à une espèce mais ou elle ne se trouve pourtant pas
espece='Lynx lynx'

col_affichage='nombreObs_normalisé_mrl'
colormap='plasma'

df_cluster_espece_manquante=chercher_zone_espece_pas_presente(df_input,df_corr_cluster,df_inpn_cluster_espece,espece,
                                      col_choice='nombreObs_normalisé_par_maille_regne')

# On groupe par 'codeMaille10Km' et on filtre ceux qui n'ont pas l'espèce
df_avec_espece = df_input[(df_input['nomScientifique']==espece)]

# On récupère la liste unique des 'codeMaille10Km'
liste_maille_avec_espece = df_avec_espece['codeMaille10Km'].unique()

df_resultat_avec=df_cluster_espece_manquante[df_cluster_espece_manquante['codeMaille10Km'].isin(liste_maille_avec_espece)]

# Afficher l'aire de répartition correspondant au cluster selectionné
seuil=max(df_resultat_avec[col_affichage])-3
seuil=0
df_filt=df_cluster_espece_manquante[df_cluster_espece_manquante[col_affichage]>=seuil]

titre='Aire de répartition potentielle de '+espece

afficher_carte_observations_espece_manquante(df_filt,df_resultat_avec,carte_maille,col_values=col_affichage,cmap_choice=colormap,color_present='black',
                                             n=1,m=1,title=titre,save_path=savepath,basemap_type='OpenStreetMap',zoom_map=7)

In [ ]:
# choix des clusters à afficher sous forme de carte

liste_cluster=[13]
#liste_cluster=list(range(1, 101))
seuil =0
var='nombreObs_normalisé_mrl'
df_filt=df_corr_cluster[df_corr_cluster['Cluster_corr'].isin(liste_cluster)]

df_filt=pd.merge(df_in,df_filt[['nomScientifique','Cluster_corr']],on='nomScientifique')
df_filt=df_filt[df_filt[var]>=seuil]
df_filt=df_filt[['codeMaille10Km','Cluster_corr']]
df_filt=df_filt.drop_duplicates()

afficher_carte_cluster(df_filt,carte_maille,col_values="Cluster_corr",cmap_choice=colormap,
                       title=titre,save_path=savepath,basemap_type='OpenStreetMap',val_alpha=0.5)

## CORRELATIONS AVEC UN SUJET

In [ ]:
# Chercher les correlations entre un sujet et un champ de recherche
clade = 'nomScientifique'
sujet='Armadillidium nasatum'

df_global=filtrer_top(df_inpn,'nombreObs',1000)
df_sujet=df_inpn[df_inpn['nomScientifique']==sujet]
df_global=pd.concat([df_sujet,df_global])
df_global = df_global.drop_duplicates()

#définition du dataframe où chercher la corrélation
methode_choice='kendall' # pearson kendall spearman 
#1000 Pearson=24s ,Kendall=29s, spearman =35s . Kendall et spearman donne à peu près les mêmes résultats, meilleurs que pearson
col_valeur='nombreObs_norm_par_maille_et_regne'
colonne_resultat = f"{col_valeur}_predit_par_correlation"

df_corr=calculer_correlation_sujet(df_global,col_valeur,clade,sujet,methode=methode_choice)

afficher_dataframe(df_corr,['Coeff_corr','nomScientifique','nomVernaculaire','genre','famille','ordre','classe','regne'],col_sort='Coeff_corr').iloc[1:10]


In [ ]:
# Fitlrer les résultats
liste=df_arbres['nomScientifique'].unique()
df_filt=df_corr[df_corr['nomScientifique'].isin(liste)]
afficher_dataframe(df_filt,['Coeff_corr','nomScientifique','nomVernaculaire','genre','famille','ordre','classe','regne'],col_sort='Coeff_corr').iloc[0:10]

In [ ]:
# Afficher la carte de l'espèce/autre la mieux corrélée
col_valeur='nombreObs_norm_par_maille_et_regne'
taxon_corr=df_corr.iloc[1, 0]
df_filt=df_inpn[(df_inpn[clade]==taxon_corr)]
titre=clade+'_'+taxon_corr+'_'+col_valeur

afficher_carte_defaut(titre,df_filt,df_geo=carte_maille,couche_sig=bioregion_gpd,col_valeur=col_valeur,cle_geo=cle_geo,quantile_inf=0,quantile_sup=0.99,
                           cmap_choice=colormap,basemap_type=fond_carte,save_path=None)

In [ ]:
# Afficher la carte de l'espèce/autre la moins corrélée
taxon_corr=df_corr.iloc[len(df_corr)-1,0]
df_filt=df_inpn[(df_inpn[clade]==taxon_corr)]
titre=clade+'_'+taxon_corr+'_'+col_valeur

afficher_carte_defaut(titre,df_filt,df_geo=carte_maille,couche_sig=departement_gpd,col_valeur=col_valeur,cle_geo=cle_geo,quantile_inf=0,quantile_sup=0.99,
                           cmap_choice=colormap,basemap_type=fond_carte,save_path=None)

In [ ]:
# Affichage de la carte de répartition prédite par corrélation 
with_specie=1 # si with_specie=0 alors ne prend pas en compte l'espece en question , si #1 alors prend en compte l'espece

#Calculer la prédiction
df_sujet_predit=recalculer_nombreObs_par_correlation(df_global,df_corr,col_valeur,sujet,with_specie) 

titre=f"Aire de répartition potentielle de {sujet}"
colormap='plasma'
n_sigma=2 #if n=0 alors seuil=0
seuil_observation=1
# Creer un seuil basé sur l'aire de répartition connue de l'espèce
df_global_avec_presence = df_global[(df_global['nomScientifique']==sujet)&(df_global['nombreObs']>=seuil_observation)]
seuil_prediction=calculer_seuil(df_global,df_sujet_predit,sujet,col_valeur,seuil_observation,n_sigma)
df_filt=df_sujet_predit[df_sujet_predit[colonne_resultat]>=seuil_prediction]


fig, ax=configurer_carte('GeoportailSatellite',13,9)
fig, ax=ajouter_couche_continue(fig, ax, df_filt,carte_maille,colonne_resultat,cle_geo,quantile_inf=0.01,quantile_sup=0.99, cmap_choice=colormap,val_alpha=0.75,colorbar_choice=True)
fig, ax=ajouter_couche_point(fig, ax,df_global_avec_presence,carte_maille,col_valeur=col_valeur,cle_geo='codeMaille10Km',color_dot='black',size_dot=2,legend_choice=True)
fig, ax=ajouter_couche_SIG(fig, ax,departement_gpd)
#fig, ax=ajouter_couche_SIG(fig, ax,departement_gpd)
ax.set_title(titre, fontsize=16)  # Taille de la police définie à 16
fig.savefig(save_path+'/'+titre+'.png', dpi=300, bbox_inches='tight')  # Enregistre au format PNG avec une résolution de 300 DPI

plt.show()

In [ ]:
# Afficher la carte de la présence recalculé via les corrélations sans prendre en compte l'espèce en question
with_specie=0 # si with_specie=0 alors ne prend pas en compte l'espece en question , si #1 alors prend en compte l'espece

#Calculer la prédiction
df_sujet_predit=recalculer_nombreObs_par_correlation(df_global,df_corr,col_valeur,sujet,with_specie) 

titre=f"Aire de répartition potentielle de {sujet}"
colormap='plasma'
n_sigma=1 #if n=0 alors seuil=0
seuil_observation=1
# Creer un seuil basé sur l'aire de répartition connue de l'espèce
df_global_avec_presence = df_global[(df_global['nomScientifique']==sujet)&(df_global['nombreObs']>=seuil_observation)]
seuil_prediction=calculer_seuil(df_global,df_sujet_predit,sujet,col_valeur,seuil_observation,n_sigma)
df_filt=df_sujet_predit[df_sujet_predit[colonne_resultat]>=seuil_prediction]


fig, ax=configurer_carte('GeoportailSatellite',13,9)
fig, ax=ajouter_couche_continue(fig, ax, df_filt,carte_maille,colonne_resultat,cle_geo,quantile_inf=0,quantile_sup=1, cmap_choice='viridis',val_alpha=0.75,colorbar_choice=True)
fig, ax=ajouter_couche_point(fig, ax,df_global_avec_presence,carte_maille,col_valeur=col_valeur,cle_geo='codeMaille10Km',color_dot='black',size_dot=2,legend_choice=True)
fig, ax=ajouter_couche_SIG(fig, ax,departement_gpd)
#fig, ax=ajouter_couche_SIG(fig, ax,departement_gpd)
ax.set_title(titre, fontsize=16)  # Taille de la police définie à 16
fig.savefig(save_path+'/'+titre+'.png', dpi=300, bbox_inches='tight')  # Enregistre au format PNG avec une résolution de 300 DPI

plt.show()


In [ ]:
# Chercher la relation entre le nombre prédit et le nombre réel
from scipy.optimize import curve_fit

# Exemple de noms de colonnes
col_values_corr='nombreObs_norm_par_maille_et_regne'  # colonne dans df_complet_predit_espece
col_values='nombreObs'
colonne_predit = f"{col_values_corr}_predit" # colonne dans df_global_avec_presence
df_global_avec_presence = df_global[(df_global['nomScientifique']==espece)]
df_merge=pd.merge(df_global_avec_presence[['codeMaille10Km','nomScientifique','nombreObs',col_values_corr]], df_complet_predit_espece[['codeMaille10Km',colonne_predit]],on='codeMaille10Km', how='outer')
df_merge = df_merge.fillna(0)
df_merge=df_merge[df_merge['nombreObs']>=1]

x = df_merge[col_values_corr].values
y = df_merge[colonne_predit].values

# Scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(x, y, alpha=0.7, color='b', label='Données')

# Appliquer l'échelle logarithmique sur les axes
plt.xscale('log')  # Échelle logarithmique sur l'axe x
plt.yscale('log')  # Échelle logarithmique sur l'axe y

# Fonction de régression de type inverse
def exponential_model(x, a, b):
    return a * np.exp(b * x)

# Fonction puissance
def power_model(x, a, b):
    return a * np.power(x, b)

# Ajustement des paramètres a et b pour la régression puissance
params, _ = curve_fit(power_model, x, y, maxfev=10000)
a, b = params

# Création des valeurs de y pour la courbe de tendance
x_range = np.linspace(min(x), max(x), 100)
y_trend = power_model(x_range, a, b)
plt.plot(x_range, y_trend, color='r', label='Courbe de tendance puissance')


# Ajout des labels et titre
plt.xlabel(col_values_corr)
plt.ylabel(colonne_predit)
plt.title(f'Scatter Plot avec regression puissance de prédit en fonction de réel')
plt.legend()
plt.grid(True)
plt.show()

print(f'Les coefficient de la loi puissance de la forme a*x^b pour {espece} sont a={round(a,2)} et b={round(b,2)}')

## PREDICTION DE LA ZONE DE PRESENCE D'ESPECES

In [ ]:
# définition du df_global
df_global=filtrer_top(df_inpn,'nombreObs',1000)

In [ ]:
# Importer la matrice de corrélation
col_values_corr='nombreObs_norm_par_maille_et_regne'

path='C:/Users/anormand/Documents/Projet Python/Biodiv/Data_intermed'

mat_corr = pd.read_csv(path+ '/mat_corr_animalvegetal_1000.csv') # ward

In [ ]:
# Calculer la matrice de corrélation
col_values_corr='nombreObs_norm_par_espece'

mat_corr=calculer_matrice_correlation(df_global,col_values_corr,m='kendall')
mat_corr= mat_corr.astype(float)

In [ ]:
# Sauvegarder les données
path='C:/Users/anormand/Documents/Projet Python/Biodiv/Data_intermed'

df_tosave = pd.DataFrame(mat_corr)
df_tosave.to_csv(path+'/mat_corr/mat_corr_AnimalVegetalFungi_1000_kendall.csv', index=False)

In [ ]:
# Completer le dataframe (global ou local)

# Si travail global
liste_codes_complet=df_global[cle_geo].unique()
listes_codes=liste_codes_complet
df_global_complet=completer_df(df_global,df_global)
df_global_complet = df_global_complet.sort_values(by=cle_ID,ascending=True)

# Si travail local
#listes_codes=listes_codes_PN_Cevennes
#df_local=df_global[(df_global['codeMaille10Km'].isin(listes_codes))]
#df_local_complet=completer_df(df_local,df_global)
#df_local_complet = df_local_complet.sort_values(by='nomScientifique',ascending=True)

# Calculer la prédiction
df_etude_complet=df_global_complet

# Jalon 1 : début du processus
start_time = time.time()

# Utilisation de la fonction
df_complet_predit=calculer_prediction(df_etude_complet,mat_corr, col_values_corr)

# Calculer et afficher le temps écoulé
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Temps écoulé: {elapsed_time:.2f} secondes")

In [ ]:
# Calculer la prédiction
df_etude_complet=df_global_complet

# Jalon 1 : début du processus
start_time = time.time()

# Utilisation de la fonction
df_complet_predit=calculer_prediction(df_etude_complet, mat_corr,col_values_corr, cle_geo='codeMaille10Km',cle_ID='cdRef')

# Calculer et afficher le temps écoulé
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Temps écoulé: {elapsed_time:.2f} secondes")

In [ ]:
# Sauvegarder le dataframe avec les prédictions
path='C:/Users/anormand/Documents/Projet Python/Biodiv/Data_intermed'

df_tosave = pd.DataFrame(df_complet_predit)
df_tosave.to_csv(path+'/df_predit/df_predit_animalvegetalfungi_1000_kendall.csv', index=False)

In [ ]:
# Importer un fichier df_complet_predit
path='C:/Users/anormand/Documents/Projet Python/Biodiv/Data_intermed'

df_complet_predit = pd.read_csv(path+ '/df_predit/df_predit_animalvegetalfungi_1000_kendall.csv') # ward

In [ ]:
# Chercher les zones les plus adaptées à une espèce mais ou elle n'a pas été recensée
espece='Sorex coronatus'
n_sigma=1#1 pour animal, 2 pour plantes
colormap='plasma'
col_values_corr='nombreObs_norm_par_maille_et_regne'
colonne_predit = f"{col_values_corr}_predit"
seuil_observation=1

# On filtre uniquement les lignes avec l'espèce concernant
df_complet_predit_espece= df_complet_predit[(df_complet_predit['nomScientifique']==espece)]

# Creer un seuil basé sur l'aire de répartition connue de l'espèce
df_global_avec_presence = df_global[(df_global['nomScientifique']==espece)&(df_global['nombreObs']>=seuil_observation)]
liste_mailles_avec_espece = df_global_avec_presence['codeMaille10Km'].unique()
df_complet_predit_avec_presence=df_complet_predit_espece[(df_complet_predit_espece['codeMaille10Km'].isin(liste_mailles_avec_espece))]
seuil_prediction=df_complet_predit_avec_presence[colonne_predit].mean()-n_sigma*df_complet_predit_avec_presence[colonne_predit].std()
df_filt=df_complet_predit_espece[df_complet_predit_espece[colonne_predit]>=seuil_prediction]

titre='Aire de répartition potentielle de '+espece

fig, ax=configurer_carte('GeoportailSatellite',13,9)
fig, ax=ajouter_couche_continue(fig, ax, df_filt,carte_maille,colonne_predit,cle_geo,quantile_inf=0,quantile_sup=0.99, cmap_choice='viridis',val_alpha=0.75,colorbar_choice=True)
fig, ax=ajouter_couche_point(fig, ax,df_global_avec_presence,carte_maille,col_valeur=col_values_corr,cle_geo='codeMaille10Km',color_dot='orange',size_dot=2,legend_choice=True)
fig, ax=ajouter_couche_SIG(fig, ax,departement_gpd)
fig, ax=ax.set_title(titre, fontsize=16)

In [ ]:
# Chercher la relation entre le nombre prédit et le nombre réel

# Exemple de noms de colonnes
col_values_corr='nombreObs_norm_par_maille_et_regne'  # colonne dans df_complet_predit_espece

colonne_predit = f"{col_values_corr}_predit" # colonne dans df_global_avec_presence
df_global_avec_presence = df_global[(df_global['nomScientifique']==espece)]
df_merge=pd.merge(df_global_avec_presence[['codeMaille10Km','nomScientifique','nombreObs',col_values_corr]], df_complet_predit_espece[['codeMaille10Km',colonne_predit]],on='codeMaille10Km', how='outer')
df_merge = df_merge.fillna(0)
df_merge=df_merge[df_merge['nombreObs']>=1]

x = df_merge[col_values_corr].values
y = df_merge[colonne_predit].values

# Scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(x, y, alpha=0.7, color='b', label='Données')

# Appliquer l'échelle logarithmique sur les axes
#plt.xscale('log')  # Échelle logarithmique sur l'axe x
#plt.yscale('log')  # Échelle logarithmique sur l'axe y

# Fonction de régression de type inverse
def exponential_model(x, a, b):
    return a * np.exp(b * x)

# Fonction puissance
def power_model(x, a, b):
    return a * np.power(x, b)

# Ajustement des paramètres a et b pour la régression puissance
params, _ = curve_fit(power_model, x, y, maxfev=10000)
a, b = params

# Création des valeurs de y pour la courbe de tendance
x_range = np.linspace(min(x), max(x), 100)
y_trend = power_model(x_range, a, b)
plt.plot(x_range, y_trend, color='r', label='Courbe de tendance puissance')


# Ajout des labels et titre
plt.xlabel(col_values_corr)
plt.ylabel(colonne_predit)
plt.title(f'Scatter Plot avec regression puissance de prédit en fonction de réel')
plt.legend()
plt.grid(True)
plt.show()

print(f'Les coefficient de la loi puissance de la forme a*x^b pour {espece} sont a={round(a,2)} et b={round(b,2)}')

## PREDICTION D'ESPECES PROBABLEMENT PRESENTES DANS UNE ZONE

In [ ]:
# Calculer la matrice de corrélation
df_global=filtrer_top(df_inpn,'nombreObs',1000)

col_values_corr='nombreObs_norm_par_maille_et_regne'
mat_corr=correlation_matrix(df_global,col_values_corr,m='kendall')
mat_corr= mat_corr.astype(float)

In [ ]:
# Sauvegarder la matrice de corrélation
path='C:/Users/anormand/Documents/Projet Python/Biodiv/Data_intermed'

df_tosave = pd.DataFrame(mat_corr)
df_tosave.to_csv(path+'/mat_corr/mat_corr_AnimalVegetal_1000_kendall.csv', index=False)

In [ ]:
# Importer la matrice de corrélation
df_global=filtrer_top(df_inpn,'nombreObs',1000)

path='C:/Users/anormand/Documents/Projet Python/Biodiv/Data_intermed/mat_corr'

mat_corr = pd.read_csv(path+ '/mat_corr_AnimalVegetalFungi_1000_kendall.csv')

In [ ]:
# Chercher les espèces qui ont le plus de chance d'être dans une zone mais qui n'ont pourtant pas été recencées
nom_parc='Vosges'
parc_gpd=departement_gpd
liste_codes=lister_mailles_dans_site(carte_maille,parc_gpd,nom_parc,taux_min=0.5,cle_geo='codeMaille10Km',cle_nom_site='nom',methode='contains')

col_values_corr='nombreObs_norm_par_maille_et_regne'

df_local=df_global[(df_global[cle_geo].isin(liste_codes))]
df_local_complet=completer_df(df_local,df_global)
df_local_complet = df_local_complet.sort_values(by=cle_ID,ascending=True)
mat_corr.index = mat_corr.columns
df_local_complet_predit=calculer_prediction(df_local_complet, mat_corr, col_values_corr)
colonne_predit = f"{col_values_corr}_predit"
df_local_especes_absentes=recherche_espece_absente(df_local_complet_predit,colonne_predit)
df_local_especes_absentes[[colonne_predit]] = df_local_especes_absentes[[colonne_predit]].apply(lambda x: round(x, 1))

afficher_dataframe(df_local_especes_absentes,[colonne_predit,'nomScientifique','nomVernaculaire','genre','famille','ordre','classe','regne','cdRef','especeProtegee']).head(20)

In [ ]:
liste_codes

## Suivi temporel de la présence des espèces

In [ ]:
# Suivi des espèces (ou autres clade) disparues entre période 1 et 2

df_filt=df_arthropodes.copy()
df_suivi_mailles = suivre_disparition_geo(df_filt,'cdRef',cle_geo)
df_suivi_mailles.head(20)
df_geo=departement_gpd

col_valeur='taux_apparue'
titre=col_valeur
colormap='YlOrRd_r'
fond_carte='GeoportailSatellite'
afficher_carte_defaut(titre,df_suivi_mailles,df_geo=df_geo,couche_sig=departement_gpd,col_valeur=col_valeur,cle_geo=cle_geo,borne_min=0,borne_max=100,
                           cmap_choice=colormap,basemap_type=fond_carte,save_path=None)

col_valeur='taux_disparue'
titre=f"Taux de disparition des espèces d'arthropodes depuis l'an 2000"
colormap='YlOrRd'
fond_carte='GeoportailSatellite'
afficher_carte_defaut(titre,df_suivi_mailles,df_geo=df_geo,couche_sig=departement_gpd,col_valeur=col_valeur,cle_geo=cle_geo,borne_min=0,
                           cmap_choice=colormap,basemap_type=fond_carte,save_path=None)

afficher_carte_defaut('nombre d especes recenceees',df_filt,df_geo=df_geo,couche_sig=departement_gpd,col_valeur='nombreObs_unique',cle_geo=cle_geo,quantile_inf=0,quantile_sup=0.99,
                           cmap_choice='viridis',basemap_type=fond_carte,save_path=None)

In [ ]:
# Afficher la carte de l'évolution de l'aire de répartition d'un taxon
taxon='Myocastor coypus'
clade='nomScientifique'
df_filt=df_inpn_periode[(df_inpn_periode[clade]==taxon)]
df_filt=df_filt[[cle_geo, clade,'periode']].drop_duplicates().reset_index(drop=True)
grouped = df_filt.groupby([cle_geo, clade])['periode'].apply(list).reset_index()

# Appliquer la fonction pour créer la colonne 'statut'
grouped['statut'] = grouped['periode'].apply(determiner_statut)
# Garder seulement les colonnes nécessaires
final_df = grouped[['codeMaille10Km', clade, 'statut']]

titre="Statut de "+taxon
            
statut_colors = {
    "Colonisation 1991-2010": "green",
    "Colonisation 2011-2024": "lightgreen",
    "Disparition 1801-1990": "red",
    "Disparition 1991-2011": "orange",
    "Présence 1991-2010 mais absent aujourd'hui": "yellow",
    "Présence continue jusqu'à aujourd'hui": "blue"
}

fig, ax=configurer_carte('GeoportailSatellite',13,9)
fig, ax=ajouter_couche_statut(fig, ax, final_df, carte_maille, statut_colors,col_valeur='statut',cle_geo='codeMaille10Km', val_alpha=0.75, legend_choice=True)
fig, ax=ajouter_couche_SIG(fig, ax,departement_gpd)
ax.set_title(titre, fontsize=16)  # Taille de la police définie à 16
fig.savefig(save_path+'/'+titre+'.png', dpi=300, bbox_inches='tight')  # Enregistre au format PNG avec une résolution de 300 DPI

plt.show()